In [6]:
import os
import sys
import json
import codecs
import importlib
import sys
sys.path.append("../")
import poses
import numpy as np
import pandas as pd
from PIL import Image

In [19]:
def log_to_df(path="robot_logs/", timestamp=0, obj=""):
    '''
    combine 3 logs:
    1. move (csv)
    2. grasp_log (json)
    3. home (csv)
    '''
    global IMAGE
    # required format
    # observation: 6x joint velocities, gripper velocity, gripper force
    # camera: 640x480x3 RGB image
    # alternative: depth image too?

    # list of dictionaries to tfds
    # https://stackoverflow.com/questions/68567630/converting-a-list-of-dictionaries-to-a-tf-dataset

    # parse out object from path
    obj = path.split('_id-')[0].split('_')[1:]
    obj = ' '.join(obj)

    # parse out timestamp from path
    if timestamp == 0:
        timestamp = path.split('_')[0].split('/')[-1]
    
    if "FAIL" in path:
        fail_reason = path.split('-')[-1]

    # load csv to pd df
    home = pd.read_csv(f"{path}/home.csv")
    move = pd.read_csv(f"{path}/move.csv")
    
    # extract 6d poses, x, y, z, rx, ry, rz from actual_TCP_pose_0 to actual_TCP_pose_5
    home_tcp = home[['actual_TCP_pose_0', 'actual_TCP_pose_1', 'actual_TCP_pose_2', 'actual_TCP_pose_3', 'actual_TCP_pose_4', 'actual_TCP_pose_5']]
    move_tcp = move[['actual_TCP_pose_0', 'actual_TCP_pose_1', 'actual_TCP_pose_2', 'actual_TCP_pose_3', 'actual_TCP_pose_4', 'actual_TCP_pose_5']]
    # calculate delta TCP pose from 'target_TCP_pose_0' to 'target_TCP_pose_5' vs 'actual_TCP_pose_0' to 'actual_TCP_pose_5'
    home_tcp_target = home[['target_TCP_pose_0', 'target_TCP_pose_1', 'target_TCP_pose_2', 'target_TCP_pose_3', 'target_TCP_pose_4', 'target_TCP_pose_5']]
    move_tcp_target = move[['target_TCP_pose_0', 'target_TCP_pose_1', 'target_TCP_pose_2', 'target_TCP_pose_3', 'target_TCP_pose_4', 'target_TCP_pose_5']]
    
    def axis_angle_to_quat(pose):
        # convert 6d pose to matrix
        tmat = poses.pose_vec_to_mtrx(pose)
        # convert rot to roll, pitch, yaw
        quat = poses.rotMatx_2_quat(tmat[:3, :3])
        quat = np.array([quat.scalar, quat.vector[0], quat.vector[1], quat.vector[2]])
        p = np.concatenate((pose[:3], quat))
        return p
    
    # very annoyed. bridge uses rpy/delta rpy
    # autolab uses quat for state, delta rpy for action. freaks!
    def axis_angle_to_rpy(pose):
        # convert 6d pose to matrix
        tmat = poses.pose_vec_to_mtrx(pose)
        # convert rot to roll, pitch, yaw
        rpy = poses.rotation_mtrx_to_rpy(tmat[:3, :3])
        # combine with translation, the first 3 elements of pose
        rpy = np.concatenate((pose[:3], rpy))
        return rpy

    # convert 6d axis-angle pose to 6d rpy pose for home_tcp and move_tcp
    home_tcp = home_tcp.apply(axis_angle_to_rpy, axis=1)
    move_tcp = move_tcp.apply(axis_angle_to_rpy, axis=1)
    home_tcp_target = home_tcp_target.apply(axis_angle_to_rpy, axis=1)
    move_tcp_target = move_tcp_target.apply(axis_angle_to_rpy, axis=1)

    home_tcp_delta = home_tcp_target - home_tcp
    move_tcp_delta = move_tcp_target - move_tcp

    # extract timestamp columns and actual_qd columns from actual_q_0 to actual_q_5
    home = home[['timestamp', 'actual_q_0', 'actual_q_1', 'actual_q_2', 'actual_q_3', 'actual_q_4', 'actual_q_5']]
    move = move[['timestamp', 'actual_q_0', 'actual_q_1', 'actual_q_2', 'actual_q_3', 'actual_q_4', 'actual_q_5']]
    # rename actual_q_0 to actual_q_5 to q0 to q5
    home.rename(columns={'actual_q_0': 'q0', 'actual_q_1': 'q1', 'actual_q_2': 'q2', 'actual_q_3': 'q3', 'actual_q_4': 'q4', 'actual_q_5': 'q5'}, inplace=True)
    move.rename(columns={'actual_q_0': 'q0', 'actual_q_1': 'q1', 'actual_q_2': 'q2', 'actual_q_3': 'q3', 'actual_q_4': 'q4', 'actual_q_5': 'q5'}, inplace=True)
    # home = home[['timestamp']]
    # move = move[['timestamp']]

    
    # assign to home and move dfs
    # home tcp has become a single column df, each cell containing a 6-element list, need to split back into 'actual_TCP_pose_0', 'actual_TCP_pose_1', 'actual_TCP_pose_2', 'actual_TCP_pose_3', 'actual_TCP_pose_4', 'actual_TCP_pose_5'
    home_tcp = pd.DataFrame(home_tcp.tolist(), columns=['x', 'y', 'z', 'rx', 'ry', 'rz'])
    move_tcp = pd.DataFrame(move_tcp.tolist(), columns=['x', 'y', 'z', 'rx', 'ry', 'rz'])
    home_tcp_delta = pd.DataFrame(home_tcp_delta.tolist(), columns=['dx', 'dy', 'dz', 'drx', 'dry', 'drz'])
    move_tcp_delta = pd.DataFrame(move_tcp_delta.tolist(), columns=['dx', 'dy', 'dz', 'drx', 'dry', 'drz'])
    print(home_tcp)
    print(type(home_tcp))

    home[['x', 'y', 'z', 'rx', 'ry', 'rz']] = home_tcp
    move[['x', 'y', 'z', 'rx', 'ry', 'rz']] = move_tcp
    home[['dx', 'dy', 'dz', 'drx', 'dry', 'drz']] = home_tcp_delta
    move[['dx', 'dy', 'dz', 'drx', 'dry', 'drz']] = move_tcp_delta

    # load json
    grasp_pth = f"{path}/grasp.json"
    obj_text = codecs.open(grasp_pth, 'r', encoding='utf-8').read()
    gl = np.array(json.loads(obj_text))
    grasp_log = pd.DataFrame(gl.tolist()) # convert to pd df
    # keep only timestamp, aperture, gripper_vel, and contact_force cols, add actual_qd columns from actual_qd_0 to actual_qd_6 from the last row of move to grasp_log

    gl_copy = grasp_log.copy()
    grasp_log = grasp_log[['timestamp', 'aperture', 'applied_force', 'contact_force']]
    # look one row ahead to get the n+1 aperture and applied_force, subtract current aperture and applied force, and set to d_aperture, d_applied_force
    grasp_log['d_aperture'] = grasp_log['aperture'].diff().shift(-1)
    grasp_log['d_applied_force'] = grasp_log['applied_force'].diff().shift(-1)
    # set the last row's d_aperture and d_applied_force to 0
    grasp_log.iloc[-1, -2:] = 0.0
    grasp_log[['q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'x', 'y', 'z', 'rx', 'ry', 'rz']] = move.iloc[-1][['q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'x', 'y', 'z', 'rx', 'ry', 'rz']]
    grasp_log[['dx', 'dy', 'dz', 'drx', 'dry', 'drz']] = 0.0
    # reorder columns so that it is: timestamp, x, y, z, qx, qy, qz, qw, dx, dy, dz, dqx, dqy, dqz, dqw, aperture, d_aperture, applied_force, d_applied_force
    grasp_log = grasp_log[['timestamp', 'q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'x', 'y', 'z', 'rx', 'ry', 'rz', 'dx', 'dy', 'dz', 'drx', 'dry', 'drz', 'aperture', 'd_aperture', 'applied_force', 'd_applied_force', 'contact_force']]
    
    # add columns to move and home dfs
    move['aperture'] = 100.0 # set to 100.0mm
    move['d_aperture'] = 0.0
    move['contact_force'] = 0.0
    move['applied_force'] = 0.0
    move['d_applied_force'] = 0.0
    home['aperture'] = grasp_log['aperture'].iloc[-1]
    home['d_aperture'] = 0.0
    home['contact_force'] = grasp_log['contact_force'].iloc[-1]
    home['applied_force'] = grasp_log['applied_force'].iloc[-1]
    home['d_applied_force'] = 0.0    

    # add task column to move, grasp_log, and home
    move_task = f"move to grasp {obj}"
    grasp_task = f"grasp {obj}"
    home_task = f"move back to original position with {obj}"
    task = f"grasp {obj} and return to original position"
    move['subtask'] = move_task
    move['task'] = task
    grasp_log['subtask'] = grasp_task
    grasp_log['task'] = task
    home['subtask'] = home_task
    home['task'] = task

    def extract_timestamp(filename):
        # extracting the float
        img = filename.split('_')[1].split('.')[:2]
        return float(f"{img[0]}.{img[1]}")

    # # get initial values, iteratively subtract 0.1s from last_move and last_home and assign to to move and home timestamps, from last to first
    move_workspace_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/workspace_img") if f.startswith("0_")])
    move_wrist_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/wrist_img") if f.startswith("0_")])
    last_move_workspace_img = move_workspace_img[-1]
    last_move_wrist_img = move_wrist_img[-1]
    last_move = max(last_move_workspace_img, last_move_wrist_img)

    home_workspace_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/workspace_img") if f.startswith("2_")])
    home_wrist_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/wrist_img") if f.startswith("2_")])
    last_home_workspace_img = home_workspace_img[-1]
    last_home_wrist_img = home_wrist_img[-1]
    last_home = max(last_home_workspace_img, last_home_wrist_img)

    for i in np.arange(len(move)-1, -1, -1):
        # iterate backwards through move timestamps
        move.loc[i, 'timestamp'] = last_move
        last_move -= 0.096
    
    for i in np.arange(len(home)-1, -1, -1):
        # iterate backwards through home timestamps
        home.loc[i, 'timestamp'] = last_home
        last_home -= 0.096

    grasp_workspace_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/workspace_img") if f.startswith("1_")])
    grasp_wrist_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/wrist_img") if f.startswith("1_")])
    
    def timestamp_to_filename(timestamp, index):
        # converting the float to a string
        return f"{index}_{str(timestamp)}.jpeg"

    move_workspace_interpolated = []
    grasp_workspace_interpolated = []
    home_workspace_interpolated = []
    move_wrist_interpolated = []
    grasp_wrist_interpolated = []
    home_wrist_interpolated = []
    # go thropugh each timestamp in move, grasp, and home
    # find the closest img timestamp and append to move_interpolated, grasp_interpolated, and home_interpolated
    for m in move['timestamp']:
        move_workspace_interpolated.append(timestamp_to_filename(min(move_workspace_img, key=lambda x:abs(x-m)), 0))
        move_wrist_interpolated.append(timestamp_to_filename(min(move_wrist_img, key=lambda x:abs(x-m)), 0))
    for g in grasp_log['timestamp']:
        grasp_workspace_interpolated.append(timestamp_to_filename(min(grasp_workspace_img, key=lambda x:abs(x-g)), 1))
        grasp_wrist_interpolated.append(timestamp_to_filename(min(grasp_wrist_img, key=lambda x:abs(x-g)), 1))
    for h in home['timestamp']:
        home_workspace_interpolated.append(timestamp_to_filename(min(home_workspace_img, key=lambda x:abs(x-h)), 2))
        home_wrist_interpolated.append(timestamp_to_filename(min(home_wrist_img, key=lambda x:abs(x-h)), 2))

    # make new columns in dataframes
    move['img'] = move_workspace_interpolated
    move['wrist_img'] = move_wrist_interpolated
    grasp_log['img'] = grasp_workspace_interpolated
    grasp_log['wrist_img'] = grasp_wrist_interpolated
    home['img'] = home_workspace_interpolated
    home['wrist_img'] = home_wrist_interpolated

    # write move, grasp_log, and home to csv files
    move.to_csv(f"{path}/0.csv", index=False)
    grasp_log.to_csv(f"{path}/1.csv", index=False)
    home.to_csv(f"{path}/2.csv", index=False)

    # load actual image, not just the filename, cast from jpeg to numpy array
    move['img'] = [np.array(Image.open(f"{path}/workspace_img/{i}")) for i in move['img']]
    move['wrist_img'] = [np.array(Image.open(f"{path}/wrist_img/{i}")) for i in move['wrist_img']]
    grasp_log['img'] = [np.array(Image.open(f"{path}/workspace_img/{i}")) for i in grasp_log['img']]
    grasp_log['wrist_img'] = [np.array(Image.open(f"{path}/wrist_img/{i}")) for i in grasp_log['wrist_img']]
    home['img'] = [np.array(Image.open(f"{path}/workspace_img/{i}")) for i in home['img']]
    home['wrist_img'] = [np.array(Image.open(f"{path}/wrist_img/{i}")) for i in home['wrist_img']]
    
    # write move, grasp_log, and home to csv files
    # move.to_csv(f"{path}/0.csv", index=False)
    # grasp_log.to_csv(f"{path}/1.csv", index=False)
    # home.to_csv(f"{path}/2.csv", index=False)

    # # combine dataframes vertically
    df = pd.concat([move, grasp_log, home], axis=0)
    # # round all float columns to 6 decimal places
    df = df.round(6)
    # # save df to csv
    # df.to_csv(f"{path}/trajectory.csv", index=False)

    # now we normalize aperture, d_aperture, applied_force, d_applied force, and contact_force
    # aperture: 0 - 100mm
    # force: 0 - 16.2N
    # normalize aperture and d_aperture
    # df['aperture'] = (df['aperture']/100.0).clip(0.0, 1.0)
    # df['d_aperture'] = (df['d_aperture']/100.0).clip(0.0, 1.0)
    # df['applied_force']   = (df['applied_force']/ 16.2).clip(0.0, 1.0)
    # df['d_applied_force'] = (df['d_applied_force']/ 16.2).clip(0.0, 1.0)
    # df['contact_force'] = (df['contact_force']/16.2).clip(0.0, 1.0)


    # return df as pickled np array
    df_pkl = df.to_numpy()
    # np.save(f"{path}/episode_{obj}_{timestamp}.npy", df_pkl)
    np.save(f"data/train/episode_{obj}_{timestamp}.npy", df_pkl)
    # np.save(f"data/fails/episode_{obj}_{timestamp}_FAIL-{fail_reason}.npy", df_pkl)

    return move, grasp_log, home

In [18]:
# fdf = log_to_df(timestamp=timestamp, obj="FIX_test", path="fix_test")
path = "1726689405.2236292_red_button_id-0"
# path = "1726689855.8823   204_yellow_block_id-0"
# iterate through all trajectory folders in path
path = "/home/wxie/workspace/deligrasp_dataset_builder/deligrasp_dataset/trajectories"
path = "fails"
for folder in os.listdir(path):
    # skip if .npy file already exists in data/train
    # obj = path.split('_id-')[0].split('_')[1:]
    # obj = ' '.join(obj)
    # timestamp = path.split('_')[0].split('/')[-1]
    # if os.path.exists(f"data/train/episode_{obj}_{timestamp}.npy"):
    #     continue
    if os.path.isdir(f"{path}/{folder}"):
        print(f"{path}/{folder}")
        move, grasp_log, home = log_to_df(path=f"{path}/{folder}")
        # print(move, grasp_log, home)
        # break
# m,g,h = log_to_df(path=path)

fails/1726692874.179009_yellow_ducky_FAIL-slip
           x         y         z        rx        ry        rz
0  -0.196780 -0.523898  0.255558 -2.596776  0.025703  3.035558
1  -0.196718 -0.523750  0.255774 -2.596932  0.025788  3.035507
2  -0.195831 -0.521694  0.259218 -2.597092  0.025784  3.035384
3  -0.193978 -0.517294  0.266678 -2.596855  0.025769  3.035175
4  -0.191150 -0.510600  0.277984 -2.596962  0.025562  3.035325
5  -0.187371 -0.501647  0.293079 -2.596956  0.025679  3.035457
6  -0.182564 -0.490439  0.312059 -2.597160  0.025856  3.035388
7  -0.177485 -0.478482  0.332264 -2.597100  0.025853  3.035446
8  -0.172394 -0.466540  0.352470 -2.597091  0.025844  3.035490
9  -0.167354 -0.454598  0.372751 -2.596858  0.025704  3.035499
10 -0.162263 -0.442661  0.392923 -2.596907  0.025776  3.035587
11 -0.157173 -0.430733  0.413197 -2.596920  0.025778  3.035492
12 -0.152082 -0.418947  0.433016 -2.597022  0.025856  3.035545
13 -0.147968 -0.409151  0.449605 -2.597167  0.025837  3.035643
14 -0.14

In [15]:
fail_reason

'grasp'

In [1]:
# build datasets
import tensorflow_datasets as tfds

# Import your custom dataset builder class
import deligrasp_dataset_dataset_builder

# Instantiate and build the dataset
builder = tfds.builder('DeligraspDataset')  # 'my_dataset' should match the class name in the Python file
builder.download_and_prepare()  # Builds the dataset, as tfds build would do via CLI

# Load the dataset
ds = builder.as_dataset()

# You can now inspect and use the dataset
print(ds)


Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\willi\tensorflow_datasets\deligrasp_dataset\incomplete.9QP3KM_1.0.0\deligrasp_dataset-train…

Generating val examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\willi\tensorflow_datasets\deligrasp_dataset\incomplete.9QP3KM_1.0.0\deligrasp_dataset-val.t…

Dataset deligrasp_dataset downloaded and prepared to C:\Users\willi\tensorflow_datasets\deligrasp_dataset\1.0.0. Subsequent calls will reuse this data.
{'train': <_PrefetchDataset element_spec={'episode_metadata': {'file_path': TensorSpec(shape=(), dtype=tf.string, name=None)}, 'steps': DatasetSpec({'action': TensorSpec(shape=(9,), dtype=tf.float64, name=None), 'discount': TensorSpec(shape=(), dtype=tf.float32, name=None), 'is_first': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_last': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_terminal': TensorSpec(shape=(), dtype=tf.bool, name=None), 'language_embedding': TensorSpec(shape=(512,), dtype=tf.float32, name=None), 'language_instruction': TensorSpec(shape=(), dtype=tf.string, name=None), 'observation': {'image': TensorSpec(shape=(480, 640, 3), dtype=tf.uint8, name=None), 'state': TensorSpec(shape=(16,), dtype=tf.float64, name=None), 'wrist_image': TensorSpec(shape=(480, 640, 3), dtype=tf.uint8, name=None)}, 'reward': Tensor

In [82]:
def extract_timestamp(filename):
    # extracting the float
    img = filename.split('_')[1].split('.')[:2]
    return float(f"{img[0]}.{img[1]}")

def timestamp_to_filename(timestamp, index):
    # converting the float to a string
    return f"{index}_{str(timestamp)}.jpeg"

def get_last_timestamps(path=""):
    # we're gonna retrieve timestamps from the image file names in the workspace_img and wrist_img folders
    # image filenames are structured as: <0,1,2>_<epoch_timestamp>.jpeg, 0,1,2 correspond to move, grasp, home respectively
    # we're gonna use the last epoch timestamp from the move and home images to assign timestamps to the move and home dataframes
    # using whichever is later, between the workspace and wrist images
    
    # for the move images, we're going to grab all the images from the path that begin with 0_
    # for the home images, we're going to grab all the images from the path that begin with 2_
    # we're going to sort the images in ascending order
    # we're going to grab the last image in the list
    # we're going to extract the epoch timestamp from the image filename
    move_workspace_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/workspace_img") if f.startswith("0_")])
    move_wrist_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/wrist_img") if f.startswith("0_")])
    last_move_workspace_img = move_workspace_img[-1]
    last_move_wrist_img = move_wrist_img[-1]
    last_move = max(last_move_workspace_img, last_move_wrist_img)

    home_workspace_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/workspace_img") if f.startswith("2_")])
    home_wrist_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/wrist_img") if f.startswith("2_")])
    last_home_workspace_img = home_workspace_img[-1]
    last_home_wrist_img = home_wrist_img[-1]
    last_home = max(last_home_workspace_img, last_home_wrist_img)

    return last_move, last_home

def interpolate_timestamps(path):
    # we are going to find the img timestamp that is closest to each row's timestamp
    move_timestamps = pd.read_csv(f"{path}/0.csv")['timestamp']
    grasp_timestamps = pd.read_csv(f"{path}/1.csv")['timestamp']
    home_timestamps = pd.read_csv(f"{path}/2.csv")['timestamp']
    move_workspace_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/workspace_img") if f.startswith("0_")])
    move_wrist_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/wrist_img") if f.startswith("0_")])
    home_workspace_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/workspace_img") if f.startswith("2_")])
    home_wrist_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/wrist_img") if f.startswith("2_")])
    grasp_workspace_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/workspace_img") if f.startswith("1_")])
    grasp_wrist_img = sorted([extract_timestamp(f) for f in os.listdir(f"{path}/wrist_img") if f.startswith("1_")])
    move_workspace_interpolated = []
    grasp_workspace_interpolated = []
    home_workspace_interpolated = []
    move_wrist_interpolated = []
    grasp_wrist_interpolated = []
    home_wrist_interpolated = []
    # go thropugh each timestamp in move, grasp, and home
    # find the closest img timestamp and append to move_interpolated, grasp_interpolated, and home_interpolated
    for m in move_timestamps:
        move_workspace_interpolated.append(timestamp_to_filename(min(move_workspace_img, key=lambda x:abs(x-m)), 0))
        move_wrist_interpolated.append(timestamp_to_filename(min(move_wrist_img, key=lambda x:abs(x-m)), 0))
    for g in grasp_timestamps:
        grasp_workspace_interpolated.append(timestamp_to_filename(min(grasp_workspace_img, key=lambda x:abs(x-g)), 1))
        grasp_wrist_interpolated.append(timestamp_to_filename(min(grasp_wrist_img, key=lambda x:abs(x-g)), 1))
    for h in home_timestamps:
        home_workspace_interpolated.append(timestamp_to_filename(min(home_workspace_img, key=lambda x:abs(x-h)), 2))
        home_wrist_interpolated.append(timestamp_to_filename(min(home_wrist_img, key=lambda x:abs(x-h)), 2))

    # make new columns in dataframes
    move['img'] = move_workspace_interpolated
    move['wrist_img'] = move_wrist_interpolated
    grasp['img'] = grasp_workspace_interpolated
    grasp['wrist_img'] = grasp_wrist_interpolated
    home['img'] = home_workspace_interpolated
    home['wrist_img'] = home_wrist_interpolated

    return move_workspace_interpolated, move_wrist_interpolated, grasp_workspace_interpolated, grasp_wrist_interpolated, home_workspace_interpolated, home_wrist_interpolated

# get_last_timestamps(path)
interpolate_timestamps(path)

    

NameError: name 'move' is not defined

In [75]:
def df_to_rlds(df, img, language_instruction="", path="robot_logs/episode.tfds", obj=""):
    episode_steps = df.to_dict(orient='records')

    # # make images array with length == len(steps) that is img at index 0 and a blank image the rest
    # images = [img] + [np.zeros_like(img) for _ in range(len(episode_steps)-1)]

    # Convert list of dictionaries to RLDS episode
    episode = {
        'steps':[{
            'timestamp': step['timestamp'],
            'action': [
                step['x'],
                step['y'],
                step['z'],
                step['rx'],
                step['ry'],
                step['rz'],
                step['aperture'],
                step['gripper_vel'],
                step['contact_force'],
                step['applied_force'],
                ],
            'observation': {
                'state': [
                step['x'],
                step['y'],
                step['z'],
                step['rx'],
                step['ry'],
                step['rz'],
                step['aperture'],
                step['gripper_vel'],
                step['contact_force'],
                step['applied_force'],
                ],
                'image': np.array(img)
            },
            'language_instruction': language_instruction,
            'reward': 0,  # Placeholder, assuming no reward data is available
            'is_terminal': False,  # Placeholder, assuming steps are not terminal
            'is_first': False,  # Placeholder, assuming steps are not terminal
            'is_last': False  # Placeholder, assuming steps are not terminal
        } for step in episode_steps]
        # } for step, i in zip(episode_steps, images)]
    }

    # Convert the episode to a list of tuples
    steps = [
        (
            step['timestamp'],     # timestep
            step['action'],       # action
            step['observation'],  # observation
            step['language_instruction'],  # language_instruction
            step['reward'],       # reward
            step['is_terminal'],   # is_terminal
            step['is_first'],   # is_terminal
            step['is_last']   # is_terminal
        )
        for step in episode['steps']
    ]

    # Generator function
    def generator():
        for step in steps:
            yield step

    # Define the dataset types and shapes
    output_types = (
        tf.float32,  # timestamp,
        tf.float32,  # action
        {'state': tf.float32, 'image': tf.uint8},  # observation
        tf.string,   # language_instruction
        tf.float32,  # reward
        tf.bool,     # is_terminal
        tf.bool,     # is_first
        tf.bool      # is_last
    )

    output_shapes = (
        tf.TensorShape([]),  # timestamp
        tf.TensorShape([10]),  # action
        {'state': tf.TensorShape([10]), 'image': tf.TensorShape([480, 640, 3])},  # observation
        tf.TensorShape([]),   # language_instruction
        tf.TensorShape([]),   # reward
        tf.TensorShape([]),   # is_terminal
        tf.TensorShape([]),   # is_first
        tf.TensorShape([])   # is_last
    )

    # Create the tf.data.Dataset
    dataset = tf.data.Dataset.from_generator(
        generator,
        output_types=output_types,
        output_shapes=output_shapes
    )

    # save tfds as file
    tf.data.Dataset.save(dataset, f'robot_logs/episode_{path}_{obj}_FIX')
    # episode = rlds.build_episode(tfds.as_numpy(dataset), metadata=output_types[0])
    # rlds.save_as_tfds(dataset, f'robot_logs/{path}.tfds')

    return episode, dataset

In [76]:
# create 480x640x3 image
img = np.random.randint(0, 255, (480, 640, 3))
rdf, tdf = df_to_rlds(fdf, img, language_instruction="FIX_test", path="fix_test", obj="FIX_test")

Instructions for updating:
Use `tf.data.Dataset.save(...)` instead.


Instructions for updating:
Use `tf.data.Dataset.save(...)` instead.


In [12]:
tfdsdf= tfds.builder_from_directory(builder_dir='robot_logs/episode_fix_test_FIX_test_FIX')


In [13]:
tfdsdf.info.features

FeaturesDict({
    'episode_metadata': FeaturesDict({
        'file_path': Text(shape=(), dtype=string),
    }),
    'steps': Dataset({
        'action': Tensor(shape=(10,), dtype=float32),
        'is_first': Scalar(shape=(), dtype=bool),
        'is_last': Scalar(shape=(), dtype=bool),
        'is_terminal': Scalar(shape=(), dtype=bool),
        'language_instruction': Text(shape=(), dtype=string),
        'observation': FeaturesDict({
            'image': Image(shape=(480, 640, 3), dtype=uint8),
            'state': Tensor(shape=(10,), dtype=float32),
        }),
        'reward': Scalar(shape=(), dtype=float32),
        'timestamp': Scalar(shape=(), dtype=float32),
    }),
})

In [17]:
d = tfdsdf.as_dataset(split='train[:1]')
episode = next(iter(d))
episode

2024-07-16 15:44:35.605904: W tensorflow/core/framework/op_kernel.cc:1839] OP_REQUIRES failed at example_parsing_ops.cc:98 : INVALID_ARGUMENT: Feature: episode_metadata/file_path (data type: string) is required but could not be found.


InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Feature: episode_metadata/file_path (data type: string) is required but could not be found.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]] [Op:IteratorGetNext] name: 

In [54]:
tfdd = tf.data.Dataset.load("robot_logs/episode_fix_test_FIX_test_FIX")

NotFoundError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} robot_logs/episode_fix_test_FIX_test_FIX/deligrasp_real_dataset-train.tfrecord-00000-of-00001; No such file or directory [Op:IteratorGetNext] name: 

In [69]:
def as_gif(images, path='temp.gif'):
  # Render the images as the gif:
  images[0].save(path, save_all=True, append_images=images[1:], duration=1000, loop=0)
  gif_bytes = open(path,'rb').read()
  return gif_bytes

# images = [step['observation']['image'] for step in episode['steps']]
# images = [Image.fromarray(image.numpy()) for image in images]
# display.Image(as_gif(images))
for step in episode['steps']:
    print(step['timestamp'])
    print(step['action'])
    print(step['observation'])
    print(step['language_instruction'])
    print(step['reward'])
    print(step['is_terminal'])
    print(step['is_first'])
    print(step['is_last'])
    print("\n")

TypeError: string indices must be integers, not 'str'

In [67]:
d

{'train': <_PrefetchDataset element_spec={'episode_metadata': {'file_path': TensorSpec(shape=(), dtype=tf.string, name=None)}, 'steps': DatasetSpec({'action': TensorSpec(shape=(10,), dtype=tf.float32, name=None), 'is_first': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_last': TensorSpec(shape=(), dtype=tf.bool, name=None), 'is_terminal': TensorSpec(shape=(), dtype=tf.bool, name=None), 'language_instruction': TensorSpec(shape=(), dtype=tf.string, name=None), 'observation': {'image': TensorSpec(shape=(480, 640, 3), dtype=tf.uint8, name=None), 'state': TensorSpec(shape=(10,), dtype=tf.float32, name=None)}, 'reward': TensorSpec(shape=(), dtype=tf.float32, name=None), 'timestamp': TensorSpec(shape=(), dtype=tf.float32, name=None)}, TensorShape([]))}>}

In [4]:
pddf = pd.read_csv("robot_logs/df_2024_07_01_18_02_32_id-0.csv")
tfdd = tf.data.Dataset.load("robot_logs/episode_2024_07_01_18_02_32_id-0")

2024-07-11 16:55:18.871150: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-07-11 16:55:18.871196: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: wxie-desktop
2024-07-11 16:55:18.871206: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: wxie-desktop
2024-07-11 16:55:18.871379: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.183.1
2024-07-11 16:55:18.871411: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 470.239.6
2024-07-11 16:55:18.871419: E external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:244] kernel version 470.239.6 does not match DSO version 535.183.1 -- cannot find working devices in this configuration


In [5]:
import tensorflow as tf
rl = tf.data.Dataset.load("robot_logs/episode_2024_07_01_18_02_32_id-0")
metadata = {
    'timestamp': tf.float32,
    'actual_qd': tf.float32,
    'aperture': tf.float32,
    'gripper_vel': tf.float32,
    'contact_force': tf.float32,
    'image': tf.uint8
}

In [6]:
episode = rlds.build_episode(rl, metadata)

In [7]:
episode

{'steps': <_LoadDataset element_spec=({'aperture': TensorSpec(shape=(), dtype=tf.float32, name=None), 'actual_qd': TensorSpec(shape=(6,), dtype=tf.float32, name=None), 'image': TensorSpec(shape=(480, 640, 3), dtype=tf.uint8, name=None), 'gripper_vel': TensorSpec(shape=(), dtype=tf.float32, name=None), 'timestamp': TensorSpec(shape=(), dtype=tf.float32, name=None), 'contact_force': TensorSpec(shape=(), dtype=tf.float32, name=None)}, TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.bool, name=None))>,
 'timestamp': tf.float32,
 'actual_qd': tf.float32,
 'aperture': tf.float32,
 'gripper_vel': tf.float32,
 'contact_force': tf.float32,
 'image': tf.uint8}

In [9]:
# convert rl, which is a tf.data.Dataset, to a tensorflow dataset, tfds, object
rltfds = tfds.as_numpy(rl)

In [12]:
for i in rltfds:
    print(i)

({'aperture': 51.68621, 'gripper_vel': 0.0, 'contact_force': 0.0, 'image': array([[[ 24,  45,  51],
        [ 23,  45,  48],
        [ 23,  45,  46],
        ...,
        [ 29,  47,  51],
        [ 28,  45,  50],
        [ 28,  45,  50]],

       [[ 24,  45,  51],
        [ 25,  47,  51],
        [ 25,  47,  48],
        ...,
        [ 26,  45,  49],
        [ 24,  43,  47],
        [ 24,  43,  47]],

       [[ 22,  44,  52],
        [ 23,  46,  51],
        [ 23,  47,  49],
        ...,
        [ 27,  46,  49],
        [ 27,  46,  50],
        [ 26,  45,  49]],

       ...,

       [[ 43,  45,  50],
        [ 43,  46,  50],
        [ 39,  46,  48],
        ...,
        [131, 138, 141],
        [133, 140, 142],
        [134, 140, 141]],

       [[ 42,  46,  48],
        [ 43,  47,  49],
        [ 41,  48,  49],
        ...,
        [128, 136, 140],
        [127, 135, 137],
        [129, 135, 137]],

       [[ 42,  48,  48],
        [ 41,  47,  47],
        [ 39,  47,  46],
        ...,

In [14]:
rltfds.element_spec

({'aperture': f32[],
  'gripper_vel': f32[],
  'contact_force': f32[],
  'image': ui8[480 640 3],
  'actual_qd': f32[6],
  'timestamp': f32[]},
 f32[],
 f32[],
 bool_[])

In [13]:
episode_tfds = rlds.build_episode(rltfds, metadata)

# pd df to tfds 

In [44]:
tfds = tf.data.Dataset.from_tensor_slices(df.to_dict(orient='list'))

In [45]:
tfds

<_TensorSliceDataset element_spec={'timestamp': TensorSpec(shape=(), dtype=tf.float32, name=None), 'actual_qd_0': TensorSpec(shape=(), dtype=tf.float32, name=None), 'actual_qd_1': TensorSpec(shape=(), dtype=tf.float32, name=None), 'actual_qd_2': TensorSpec(shape=(), dtype=tf.float32, name=None), 'actual_qd_3': TensorSpec(shape=(), dtype=tf.float32, name=None), 'actual_qd_4': TensorSpec(shape=(), dtype=tf.float32, name=None), 'actual_qd_5': TensorSpec(shape=(), dtype=tf.float32, name=None), 'aperture': TensorSpec(shape=(), dtype=tf.float32, name=None), 'gripper_vel': TensorSpec(shape=(), dtype=tf.float32, name=None), 'contact_force': TensorSpec(shape=(), dtype=tf.float32, name=None)}>

# pd df direct to rlds

In [61]:
# !pip install rlds
import rlds

episode_steps = df.to_dict(orient='records')

# Convert list of dictionaries to RLDS episode
episode = {
    'steps':[{
        'observation': {
            'timestamp': step['timestamp'],
            'actual_qd': [
                step['actual_qd_0'],
                step['actual_qd_1'],
                step['actual_qd_2'],
                step['actual_qd_3'],
                step['actual_qd_4'],
                step['actual_qd_5']
            ],
            'aperture': step['aperture'],
            'gripper_vel': step['gripper_vel'],
            'contact_force': step['contact_force'],
            'image': np.zeros([64, 64, 3])
        },
        'action': 1,  # Placeholder, assuming no action data is available
        'reward': 0,  # Placeholder, assuming no reward data is available
        'is_terminal': False  # Placeholder, assuming steps are not terminal
    } for step in episode_steps]
}

# Print the RLDS episode
print(episode)


{'steps': [{'observation': {'timestamp': 1719530032.843785, 'actual_qd': [0.001078, -2.469845, 2.349459, -1.955513, -1.612564, -3.034139], 'aperture': 27.80900566920235, 'gripper_vel': 0.0, 'contact_force': 0.0, 'image': array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       ...,

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [

In [62]:
# Convert the episode to a list of tuples
steps = [
    (
        step['observation'],  # observation
        step['action'],       # action
        step['reward'],       # reward
        step['is_terminal']   # is_terminal
    )
    for step in episode['steps']
]

# Generator function
def generator():
    for step in steps:
        yield step

# Define the dataset types and shapes
output_types = (
    {
        'timestamp': tf.float32,
        'actual_qd': tf.float32,
        'aperture': tf.float32,
        'gripper_vel': tf.float32,
        'contact_force': tf.float32,
        'image': tf.uint8
    },
    tf.float32,  # action
    tf.float32,  # reward
    tf.bool      # is_terminal
)

output_shapes = (
    {
        'timestamp': tf.TensorShape([]),
        'actual_qd': tf.TensorShape([6]),
        'aperture': tf.TensorShape([]),
        'gripper_vel': tf.TensorShape([]),
        'contact_force': tf.TensorShape([]),
        'image': tf.TensorShape([64, 64, 3])
    },
    tf.TensorShape([]),  # action
    tf.TensorShape([]),  # reward
    tf.TensorShape([])   # is_terminal
)

# Create the tf.data.Dataset
dataset = tf.data.Dataset.from_generator(
    generator,
    output_types=output_types,
    output_shapes=output_shapes
)

# Print the dataset
for element in dataset:
    print(element)



({'timestamp': <tf.Tensor: shape=(), dtype=float32, numpy=1719530000.0>, 'actual_qd': <tf.Tensor: shape=(6,), dtype=float32, numpy=
array([ 1.078000e-03, -2.469845e+00,  2.349459e+00, -1.955513e+00,
       -1.612564e+00, -3.034139e+00], dtype=float32)>, 'aperture': <tf.Tensor: shape=(), dtype=float32, numpy=27.809006>, 'gripper_vel': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'contact_force': <tf.Tensor: shape=(), dtype=float32, numpy=0.0>, 'image': <tf.Tensor: shape=(64, 64, 3), dtype=uint8, numpy=
array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],


In [8]:
import json
path = "robot_logs/grasp_logs.json"

# get content after last newline of stdout
with open(path, 'w') as f:
    json.dump(gl.tolist(), f)



In [9]:
import codecs
obj_text = codecs.open(path, 'r', encoding='utf-8').read()
b_new = json.loads(obj_text)
a_new = np.array(b_new)